In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

import os
import pathlib as pl

In [ ]:
def pretty_ax(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(
        axis='both',  
        which='both',      
        bottom=True,     
        top=False,
        left=False,
        labelbottom=True,
        labelleft = True)
    ax.spines["bottom"].set_linewidth(1.5)
    ax.spines["left"].set_linewidth(1.5)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

def compute_residuals(df: pd.DataFrame, gene_list: np.ndarray, deg: int, abso: bool=True):
    train_df = df.drop(gene_list)
    test_df = df.loc[gene_list]
    
    x_train = train_df.Bulk.ravel()
    y_train = train_df.Bulkified.ravel()
    
    x_test = test_df.Bulk.ravel()
    y_test = test_df.Bulkified.ravel()
    
    pr = PolynomialFeatures(degree = deg)
    polyfit = pr.fit(x_train.reshape(-1,1))
    
    X_train = polyfit.transform(x_train.reshape(-1,1))
    X_test = polyfit.transform(x_test.reshape(-1,1))
    
    lr_2 = LinearRegression()
    lr_2.fit(X_train, y_train)
    
    y_train_pred = lr_2.predict(X_train)
    y_pred = lr_2.predict(X_test)
    if abso:
        abs_res = pd.Series(y_test - y_pred).abs()
    else:
        abs_res = pd.Series(y_test - y_pred)
        
    return abs_res, r2_score(y_train, y_train_pred)

In [ ]:
def get_value_patient(adata: sc.AnnData, logbulk: pd.DataFrame, patient: str, 
                      deg: int, col_name: str, n_rep: int=500, layer_name: str= "counts", 
                      n_closest: int=500, abso: bool=True):
    
    cells = adata.obs[adata.obs[col_name]==patient].index

    patadata = adata[cells].copy()

    counts = patadata.layers[layer_name].toarray()

    if layer_name=="TPM":
        bulkified = pd.DataFrame(counts.mean(axis=0),index=patadata.var_names)
    else:
        bulkified = pd.DataFrame(counts.sum(axis=0),index=patadata.var_names)
    bulkified = bulkified.applymap(np.log1p)

    df = pd.concat([bulkified, logbulk.loc[patient]],axis=1).dropna()
    df.columns = ["Bulkified","Bulk"]

    mt_genes = df.index[df.index.str.startswith("MT-")].to_numpy()
    
    mt_res, _ = compute_residuals(df, gene_list=mt_genes, deg=deg, abso=abso)
    
    mt_mean = mt_res.mean()

    sorted_df = df.sort_values("Bulkified")
    sorted_df = sorted_df.drop(mt_genes)

    pool_genes = sorted_df.iloc[sorted_df.shape[0]-n_closest:sorted_df.shape[0]].index

    rng = np.random.default_rng(42)
    ctrl_res = []
    full_ctrl_dist = []
    for i in range(n_rep):
        ctrl_genes = rng.choice(pool_genes, size=(len(mt_genes)), replace=False)
        res, _ = compute_residuals(df, gene_list=ctrl_genes, deg=deg, abso=abso)
        ctrl_res.append(res.mean())
        full_ctrl_dist.append(res.ravel())

    p_val = np.sum(ctrl_res>=mt_mean)/n_rep
    
    return mt_mean, mt_res, p_val, ctrl_res, np.hstack(full_ctrl_dist)

# Breast Wu

In [ ]:
bulk = pd.read_csv("/add/path/here/auxiliary_data/BRCA_Wu_fpkm.csv",index_col=0).T

In [ ]:
logbulk = bulk.applymap(np.log1p)

In [ ]:
adata = sc.read_h5ad("/add/path/here/filtered_data/Breast_Wu_10X/filtered_adata.h5ad")

In [ ]:
adata.var_names_make_unique()

adata = adata[:,(adata.var.mean_counts>=0.01)].copy()

In [ ]:
for deg in tqdm(np.arange(1,10)):
    means, pvals, ctrl_dist = {},{},{}
    for pat in tqdm(logbulk.index):
        mt_mean, _, p_val, ctrl_res, _ = get_value_patient(adata=adata, logbulk=logbulk, patient=pat, 
                          deg=deg, col_name="Patient", n_rep=500, n_closest=500, abso=False)
        means[pat] = [mt_mean]
        pvals[pat] = [p_val]
        ctrl_dist[pat] = ctrl_res

    ctrl_dist = pd.DataFrame(ctrl_dist).T

    means = pd.DataFrame(means).T

    pvals = pd.DataFrame(pvals).T
    
    resdir = pl.Path(f"/add/path/here/results_bulk_vs_bulkified/Breast_Wu/deg{deg}")
    os.makedirs(resdir, exist_ok=True)

    ctrl_dist.to_csv(resdir / "ctrl_dist.csv")
    means.to_csv(resdir / "mtmeans.csv")
    pvals.to_csv(resdir / "pvals.csv")

# Compute plot

In [ ]:
mean_dir = pl.Path("/add/path/here/results_bulk_vs_bulkified/Breast_Wu/")

pvals_full = {}
for f in mean_dir.iterdir():
    if f.stem  not in [".DS_Store"]:
        pvals_full[f.stem] = pd.read_csv(f / "pvals.csv", index_col=0)


ctrl_res = {}
for f in mean_dir.iterdir():
    if f.stem  not in [".DS_Store"]:
        ctrl_res[f.stem] = pd.read_csv(f / "ctrl_dist.csv", index_col=0)


mt_means = {}
for f in mean_dir.iterdir():
    if f.stem  not in [".DS_Store"]:
        mt_means[f.stem] = pd.read_csv(f / "mtmeans.csv", index_col=0)


for deg in [f"deg{i}" for i in np.arange(1,7)]:

    plot_df = mt_means[deg].drop("CID4461")
    plot_df = plot_df.reset_index()
    plot_df.columns = ["Patient","MT res. means"]
    
    text_ps = pvals_full[deg].loc[plot_df.Patient.ravel()].applymap(lambda x: "" if x>=0.05 else ("*" if 0.01<=x<0.05 else "**")).values.ravel()

    qt = ctrl_res[deg].loc[plot_df.Patient.ravel()]

    y1, y2 = qt.quantile(0.05,axis=1).ravel(),qt.quantile(0.95,axis=1).ravel()

    fig, ax = plt.subplots(1,1,figsize=(7,2))
    sns.scatterplot(data=plot_df, x="Patient", y="MT res. means",ax=ax)
    pretty_ax(ax)
    ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='right')
    ax.hlines(y=0, xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], linestyle="--", color="gray")
    ax.fill_between(np.arange(plot_df.shape[0]), y1, y2, color="grey", alpha=0.25)
    ax.set_xlabel("")
    
    for i,p in enumerate(text_ps):
        ax.text(i, ax.get_ylim()[1], p, ha='center', va='center')
    fig.savefig(f"/add/path/here/figures/bulk_vs_bulkified/Breast_Wu_deg{deg}.svg", dpi=200, 
                bbox_inches="tight")

# Compute average R2 scores

In [ ]:
layer_name = "counts"
col_name = "Patient"

scores = {}
for patient in tqdm(logbulk.index.intersection(adata.obs[col_name].unique())):
    pat_scores = {}
    for deg in np.arange(1,8):
        cells = adata.obs[adata.obs[col_name]==patient].index

        patadata = adata[cells].copy()

        counts = patadata.layers[layer_name].toarray()

        if layer_name=="TPM":
            bulkified = pd.DataFrame(counts.mean(axis=0),index=patadata.var_names)
        else:
            bulkified = pd.DataFrame(counts.sum(axis=0),index=patadata.var_names)
        bulkified = bulkified.applymap(np.log1p)

        df = pd.concat([bulkified, logbulk.loc[patient]],axis=1).dropna()
        df.columns = ["Bulkified","Bulk"]

        mt_genes = df.index[df.index.str.startswith("MT-")].to_numpy()

        res, score = compute_residuals(df, gene_list=mt_genes, deg=deg)

        mt_mean = res.mean()

        pat_scores[deg] = [score]
    scores[patient] = pd.DataFrame(pat_scores)

In [ ]:
avg_r2 = pd.concat(scores).reset_index().drop("level_1",axis=1).set_index("level_0").mean()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(2,1))
sns.scatterplot(data=avg_r2.reset_index(), x="index", y=0,ax=ax)
pretty_ax(ax)
ax.set_ylabel("R2")
ax.set_xlabel("Degree")
fig.savefig("/add/path/here/figures/bulk_vs_bulkified/Breast_Wu_R2.svg", dpi=200, 
                bbox_inches="tight")

# Breast Chung

In [ ]:
bulk = pd.read_csv("/add/path/here/auxiliary_data/Breast_Chung_bulk.csv",index_col=0)

In [ ]:
logbulk = bulk.applymap(np.log1p)

In [ ]:
logbulk = logbulk.loc[:,~logbulk.columns.duplicated()]

In [ ]:
adata = sc.read_h5ad("/add/path/here/filtered_data/Breast_Chung")

In [ ]:
adata.var_names_make_unique()

adata = adata[:,(adata.var.mean_counts>=0.1)].copy()

In [ ]:
for deg in tqdm(np.arange(1,10)):
    means, pvals, ctrl_dist = {},{},{}
    for pat in tqdm(logbulk.index.intersection(adata.obs["sample"].unique())):
        mt_mean, _, p_val, ctrl_res, _ = get_value_patient(adata=adata, logbulk=logbulk, patient=pat, 
                            col_name="sample",
                          deg=deg, n_rep=500, layer_name="TPM", n_closest=500, abso=False)
        means[pat] = [mt_mean]
        pvals[pat] = [p_val]
        ctrl_dist[pat] = ctrl_res

    ctrl_dist = pd.DataFrame(ctrl_dist).T

    means = pd.DataFrame(means).T

    pvals = pd.DataFrame(pvals).T
    
    resdir = pl.Path(f"/add/path/here/results_bulk_vs_bulkified/Breast_Chung/deg{deg}")
    os.makedirs(resdir, exist_ok=True)

    ctrl_dist.to_csv(resdir / "ctrl_dist.csv")
    means.to_csv(resdir / "mtmeans.csv")
    pvals.to_csv(resdir / "pvals.csv")

In [ ]:
mean_dir = pl.Path("/add/path/here/results_bulk_vs_bulkified/Breast_Chung/")

pvals_full = {}
for f in mean_dir.iterdir():
    if f.stem  not in [".DS_Store"]:
        pvals_full[f.stem] = pd.read_csv(f / "pvals.csv", index_col=0)


ctrl_res = {}
for f in mean_dir.iterdir():
    if f.stem  not in [".DS_Store"]:
        ctrl_res[f.stem] = pd.read_csv(f / "ctrl_dist.csv", index_col=0)


mt_means = {}
for f in mean_dir.iterdir():
    if f.stem  not in [".DS_Store"]:
        mt_means[f.stem] = pd.read_csv(f / "mtmeans.csv", index_col=0)


for deg in [f"deg{i}" for i in np.arange(1,7)]:

    plot_df = mt_means[deg]
    plot_df = plot_df.reset_index()
    plot_df.columns = ["Patient","MT res. means"]
    
    text_ps = pvals_full[deg].loc[plot_df.Patient.ravel()].applymap(lambda x: "" if x>=0.05 else ("*" if 0.01<=x<0.05 else "**")).values.ravel()

    qt = ctrl_res[deg].loc[plot_df.Patient.ravel()]

    y1, y2 = qt.quantile(0.05,axis=1).ravel(),qt.quantile(0.95,axis=1).ravel()

    fig, ax = plt.subplots(1,1,figsize=(3.5,2))
    sns.scatterplot(data=plot_df, x="Patient", y="MT res. means",ax=ax)
    pretty_ax(ax)
    ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='right')
    ax.hlines(y=0, xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], linestyle="--", color="gray")
    ax.fill_between(np.arange(plot_df.shape[0]), y1, y2, color="grey", alpha=0.25)
    ax.set_xlabel("")
    
    for i,p in enumerate(text_ps):
        ax.text(i, ax.get_ylim()[1], p, ha='center', va='center')
    fig.savefig(f"/add/path/here/figures/bulk_vs_bulkified/Breast_Chung_deg{deg}.svg", dpi=200, 
                bbox_inches="tight")

# Compute average R2 scores

In [ ]:
layer_name = "TPM"
col_name = "sample"

scores = {}
for patient in tqdm(logbulk.index.intersection(adata.obs[col_name].unique())):
    pat_scores = {}
    for deg in np.arange(1,8):
        cells = adata.obs[adata.obs[col_name]==patient].index

        patadata = adata[cells].copy()

        counts = patadata.layers[layer_name].toarray()

        if layer_name=="TPM":
            bulkified = pd.DataFrame(counts.mean(axis=0),index=patadata.var_names)
        else:
            bulkified = pd.DataFrame(counts.sum(axis=0),index=patadata.var_names)
        bulkified = bulkified.applymap(np.log1p)

        df = pd.concat([bulkified, logbulk.loc[patient]],axis=1).dropna()
        df.columns = ["Bulkified","Bulk"]

        mt_genes = df.index[df.index.str.startswith("MT-")].to_numpy()

        res, score = compute_residuals(df, gene_list=mt_genes, deg=deg)

        mt_mean = res.mean()

        pat_scores[deg] = [score]
    scores[patient] = pd.DataFrame(pat_scores)

In [ ]:
avg_r2 = pd.concat(scores).reset_index().drop("level_1",axis=1).set_index("level_0").mean()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(2,1))
sns.scatterplot(data=avg_r2.reset_index(), x="index", y=0,ax=ax)
pretty_ax(ax)
ax.set_ylabel("R2")
ax.set_xlabel("Degree")
fig.savefig("/add/path/here/figures/bulk_vs_bulkified/Breast_Chung_R2.svg", dpi=200, 
                bbox_inches="tight")

# Pick the model

In [ ]:
mean_dir = pl.Path("/add/path/here/results_bulk_vs_bulkified/Breast_Chung/")

ctrl_res = {}
for f in mean_dir.iterdir():
    if f.stem!=".DS_Store":
        ctrl_res[f.stem] = pd.read_csv(f / "ctrl_dist.csv", index_col=0)

full_res = pd.concat([ctrl_res[dg].mean(axis=1) for dg in ctrl_res],axis=1)

mean_res = full_res.mean().round(2)

In [ ]:
mean_res

In [ ]:
mean_res.argsort()+1

In [ ]:
mean_dir = pl.Path("/add/path/here/results_bulk_vs_bulkified/Breast_Wu/")

ctrl_res = {}
for f in mean_dir.iterdir():
    if f.stem!=".DS_Store":
        ctrl_res[f.stem] = pd.read_csv(f / "ctrl_dist.csv", index_col=0)

full_res = pd.concat([ctrl_res[dg].mean(axis=1) for dg in ctrl_res],axis=1)

mean_res = full_res.mean().round(2)

In [ ]:
mean_res

In [ ]:
full_res.mean().argsort()+1